In [1]:
import sys
PATH_TO_PROJECT = "/home/tanawatp/Documents/Hexapod/PhysicalAI_modelling"
sys.path.append(PATH_TO_PROJECT)
from UtilsFunction.UtilsFunc import*
from UtilsFunction.KinematicsFunc import*
from pydrake.geometry import StartMeshcat
from pydrake.visualization import ModelVisualizer
from pydrake.math import RigidTransform, RotationMatrix

In [ ]:
plant,robot_ind = DefinePlant(model_name="Hexapod_mockup",
                    is_fixed=True,
                    fixed_frame="BaseLink")
# Start Meshcat
print("Click on link above to open Meshcat visualization.")
meshcat = StartMeshcat()
meshcat.SetProperty("/drake/visualizer", "visible", False)

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click on link above to open Meshcat visualization


In [ ]:
# Task space parameters
step_length = 0.0 # m
desired_base_height = 0.1 # m
desired_leg_length = 0.45 # m


### Calc end effector position w.r.t. base frame

In [ ]:
# Add your code to find eff_P_base here


### Solve inver kinematics

In [ ]:
IK_solver = InverseKinematics(plant)
end_effector_names = [
                      "front_left_Endeffector",  "front_right_Endeffector",
                      "middle_left_Endeffector", "middle_right_Endeffector",
                      "rear_left_Endeffector",   "rear_right_Endeffector"]
base_frame = plant.GetFrameByName("BaseLink")
# Add position constraints
for i in range(6):
    end_effector_frame = plant.GetFrameByName(end_effector_names[i])
    eff_P_base = ... # Fill the value here with np.array
    IK_solver.AddPositionConstraint(
        frameB=end_effector_frame,
        p_BQ=np.zeros(3),     # point in end-effector frame
        frameA=base_frame,
        p_AQ_lower=eff_P_base-0.001,
        p_AQ_upper=eff_P_base+0.001
    )
q_variables = IK_solver.q()
# Add joint limits
# for i in range(plant.num_positions()):
#     IK_solver.prog().AddBoundingBoxConstraint(-2.0*np.pi/3.0, 2.0*np.pi/3.0, q_variables[i])
initial_guess = np.ones(plant.num_positions())*0.1
result = Solve(IK_solver.prog(),initial_guess)
q = None
if result.is_success():
    q_solution = result.GetSolution(q_variables)
    print(q_solution)
    q = q_solution

[ 0.2006561  -0.53674354  0.21080336 -0.29092044 -0.16150708 -0.38253035
  0.12013297  0.18552473 -0.95637244  0.12013297  0.18552473 -0.95637244
 -0.29032779 -0.16150708 -0.38253035  0.2006561  -0.53674354  0.21080336]


In [6]:
ListJointsName(plant=plant,model_instance_index=robot_ind)

GetJointIndices:  [JointIndex(0), JointIndex(1), JointIndex(2), JointIndex(3), JointIndex(4), JointIndex(5), JointIndex(6), JointIndex(7), JointIndex(8), JointIndex(9), JointIndex(10), JointIndex(11), JointIndex(12), JointIndex(13), JointIndex(14), JointIndex(15), JointIndex(16), JointIndex(17), JointIndex(18), JointIndex(19), JointIndex(20), JointIndex(21), JointIndex(22), JointIndex(23), JointIndex(24), JointIndex(25), JointIndex(26)]
Joint JointIndex(0): base_x_axis_offset
Joint JointIndex(1): base_z_axis_offset
Joint JointIndex(2): front_left_HeelJoint
Joint JointIndex(3): front_left_HipJoint
Joint JointIndex(4): front_left_KneeJoint
Joint JointIndex(5): front_left_Endeffector
Joint JointIndex(6): middle_left_HeelJoint
Joint JointIndex(7): middle_left_HipJoint
Joint JointIndex(8): middle_left_KneeJoint
Joint JointIndex(9): middle_left_Endeffector
Joint JointIndex(10): rear_left_HeelJoint
Joint JointIndex(11): rear_left_HipJoint
Joint JointIndex(12): rear_left_KneeJoint
Joint JointI

### IK solution check

In [7]:
P = CalcfForwaredKinematics(plant=plant,
                            joint_angles=q,
                            base_frame_name="BaseLink",
                            end_effector_frame_name="middle_left_Endeffector")
print("Desired middle_left_Endeffector positions:\n", [step_lenght*0.5,desired_leg_lenght,-desired_base_heigh])
print("middle_left_Endeffector positions from IK:\n",P)

Desired middle_left_Endeffector positions:
 [0.1, 0.45, -0.1]
middle_left_Endeffector positions from IK:
 [ 0.099  0.449 -0.099]


### Visualization

In [8]:
# Create ModelVisualizer 
visualizer = ModelVisualizer(meshcat=meshcat,
                             visualize_frames=True,
                             triad_length=0.05,
                             triad_radius=0.001)
urdf_name = "Hexapod_mockup.urdf"
robot_urdf_path = f"{PATH_TO_PROJECT}/Models/{urdf_name}"
robot = visualizer.parser().AddModels(robot_urdf_path)
plant_viz = visualizer.parser().plant()
robot_body_frame = plant_viz.GetFrameByName("BaseLink")
init_pos = [0.0,0.0,0.0] 
init_orien = np.asarray([0, 0, 0])
X_floorRobot = RigidTransform(
    RollPitchYaw(init_orien * np.pi / 180), p=[0.0,0.0,0.0])
plant_viz.WeldFrames(plant_viz.world_frame(),robot_body_frame, X_floorRobot)
visualizer.Finalize()
# Create context
context = plant_viz.CreateDefaultContext()
if result.is_success():
    visualizer.Run(position=q,loop_once=False)
else:
    visualizer.Run(loop_once=False)


INFO:drake:Click 'Stop Running' or press Esc to quit
